# 1. Preparation

### Import libraries

In [ ]:
import os
import math
import datetime
import modelling
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

### Import data

In [ ]:
tickers = ['XLF', 'XLK', 'XLE', 'XRT']
path = r'C:\Users\JD\Google Drive\Quantitative Trading\Data'
dfs = [pd.read_csv(os.path.join(path, ticker + '.csv'), index_col=0, parse_dates=[0], dayfirst=True) for ticker in tickers]
for ticker, df in zip(tickers, dfs):
    df.columns = [ticker + ' ' + col for col in df.columns]

### Pre-processing

Merge data:

In [ ]:
dataset = modelling.aggregate(dfs)
datasetClean = dataset.dropna()
datasetClean.head()

### Inspecting dataset

In [ ]:
datasetClean.plot(y=[ticker + ' Adj Close' for ticker in tickers])

# 2. Modelling

Derive mean return:

In [ ]:
datasetCopy = datasetClean.copy()
adjClsCols = [ticker + ' Adj Close' for ticker in tickers]
logRetCols = [ticker + ' Log Ret' for ticker in tickers]
for logRetCol, adjClsCol in zip(logRetCols, adjClsCols):
    datasetCopy[logRetCol] = np.log(datasetCopy[adjClsCol].pct_change(1) + 1)
datasetCopy['Mean Log Ret'] = datasetCopy[logRetCols].mean(axis=1)
datasetCopy = datasetCopy.dropna()
datasetCopy.head()

Obtain excess returns through demeaning:

In [ ]:
xcessRetCols = [ticker + ' Excess Log Ret' for ticker in tickers]
for xcessRetCol, logRetCol in zip(xcessRetCols, logRetCols):
    datasetCopy[xcessRetCol] = datasetCopy[logRetCol] - datasetCopy['Mean Log Ret']
datasetCopy.head()

Derive weights:

In [ ]:
datasetCopy['Sum Excess Log Rets'] = datasetCopy[xcessRetCols].abs().sum(axis=1)
weightCols = [ticker + ' Weight' for ticker in tickers]
for weightCol, xcessRetCol in zip(weightCols, xcessRetCols):
    datasetCopy[weightCol] = datasetCopy[xcessRetCol] / datasetCopy['Sum Excess Log Rets']
datasetCopy.head()

# 3. Backtesting

In [ ]:
initialCapital = 10000
clsCols = [ticker + ' Close' for ticker in tickers]
allocationCols = [ticker + ' Allocated' for ticker in tickers]
capitalUsedCols = [ticker + ' Positions To Take' for ticker in tickers]
cashLeftCol = 'Cash Left'
navCol = 'NAV'

datasetCopy[navCol] = initialCapital
datasetCopy[cashLeftCol] = 0
for idx in datasetCopy.index:
    for allocationCol, capitalUsedCol, clsCol, weightCol in zip(allocationCols, capitalUsedCols, clsCols, weightCols):
        datasetCopy.loc[idx, allocationCol] = datasetCopy.loc[idx, navCol] * datasetCopy.loc[idx, weightCol]
        datasetCopy.loc[idx, capitalUsedCol] = datasetCopy.loc[idx, allocationCol] // datasetCopy.loc[idx, clsCol]
        
        if idx == datasetCopy.index[0]:
            datasetCopy.loc[idx, cashLeftCol] += datasetCopy.loc[idx, allocationCol] - datasetCopy.loc[idx, capitalUsedCol]
        else: